In [1]:
rm(list=ls()) #把当前环境中的对象全部删除

.libPaths()
#.libPaths("D:/software/R-3.5.2/library")
library(gurobi)
library(slam)
library(Matrix)

[1] "D:/software/R-4.2.0/library"

Loading required package: slam



> Umatf(2,2)
4 x 4 sparse Matrix of class "dgCMatrix"
              
[1,]  1 .  . .
[2,] -1 1  . .
[3,]  . .  1 .
[4,]  . . -1 1

In [2]:
Umatf=function(Tmax,W)
{
  i1=1:(Tmax*W)
  j1=i1
  ii=2:Tmax
  i2=ii
  for(w in 2:W)
  {
    i2=c(i2,ii+(w-1)*Tmax)
  }
  j2=i2-1
  i=c(i1,i2)
  j=c(j1,j2)
  x=c(rep(1,Tmax*W),rep(-1,(Tmax-1)*W))
  Umat<- sparseMatrix(i, j, x=x,dims=c(Tmax*W,Tmax*W))
  return(Umat)
} 
# i是1,2,...,Tmax*W,2,3,...,W*Tmax
# j是1,2,...,Tmax*W,1,2,...,W*Tmax-1
# U是对角线上是1，对角线左边是-1的Tmax*W by Tmax*W
# when W=2
# [U^0 0]
# [0 U^0]

> U1matf(2,2)
     [,1] [,2] [,3] [,4]
[1,]   -1    0    0    0
[2,]    0   -1    0    0
[3,]    0    0   -1    0
[4,]    0    0    0   -1

In [2]:
U1matf=function(Tmax,W)
{
  Umat=diag(x=-1,Tmax*W,Tmax*W)
  return(Umat)
}
# -U^1

> U2matf(2,2)
4 x 4 sparse Matrix of class "dgCMatrix"
            
[1,] . . . .
[2,] 1 . . .
[3,] . . . .
[4,] . . 1 .

In [2]:
U2matf=function(Tmax,W)
{
  x=2:(Tmax*W)
  y=1:(Tmax*W-1)
  val=rep(1,Tmax*W-1)
  Umat=sparseMatrix(i=x,j=y,x=val,dims = c(Tmax*W,Tmax*W))
  for (w in 1:(W-1)) {
    Umat[(w-1)*Tmax+(Tmax+1),(w-1)*Tmax+Tmax]=0
  }
  return(Umat)
}
# -U^2

> Ematf(2,2)
2 x 4 sparse Matrix of class "dgCMatrix"
            
[1,] 1 1 . .
[2,] . . 1 1

In [2]:
Ematf=function(Nbar,T)
{
  i=c()
  j=c()
  x=c()
  for(n in 1:Nbar)
  {
    for(t in 1:T)
    {
      i=c(i,n)
      j=c(j,(n-1)*T+t)
      x=c(x,1)
    }
  }
  # i是1,1,1,...(T个),2,2,2,...,Nbar,Nbar,...,Nbar; 
  # j是1,2,3,...,T,T+1,T+2,...,...,T*(Nbar-1)+1,...T*Nbar
  # x是1,1,...1 (Nbar*T个)
  Emat<- sparseMatrix(i, j, x=x,dims=c(Nbar,T*Nbar))
  return(Emat)
} #E乘z，小于等于e，就是每个zit对t求和是1，z是按i排序，然后按t

> E2matf(2,2)
2 x 4 sparse Matrix of class "dgCMatrix"
            
[1,] 1 . 1 .
[2,] . 1 . 1

In [2]:
E2matf=function(Nbar,T)
{
  i=c()
  j=c()
  x=c()
  for(t in 1:T)
  {
    for(n in 1:Nbar)
    {
      i=c(i,t)
      j=c(j,(n-1)*T+t)
      x=c(x,1)
    }
  }
  Emat<- sparseMatrix(i, j, x=x,dims=c(T,T*Nbar))
  return(Emat)
  #i是1,1,1,...,2,2,...,T,T,...,T(各Nbar个)
  #j是1,T+1,2T+1,...,2,T+2,...,(Nbar-1)*T+2,...,T,T+T,...,Nbar*T
} #E2 是T by T*Nbar

#ps: possibility of show up
ps=rep(0.8,T) #T=2 [1] 0.8 0.8
> Mmatnsf(2,2,2,2,ps,1)
     [,1] [,2] [,3] [,4]
[1,]    1    0    1    0
[2,]    0    1    0    1
[3,]    1    0    1    0
[4,]    0    0    0    0
> Mmatnsf(2,2,2,2,ps,10)
     [,1] [,2] [,3] [,4]
[1,]    1    0    1    0
[2,]    0    1    0    1
[3,]    1    0    1    0
[4,]    0    1    0    1

In [2]:
Mmatnsf=function(Tmax,WW,Nbar,TT,ps,seed)
{
  set.seed(seed)
  
  Mmat=matrix(0,Tmax*WW,Nbar*TT)
  
  #seed=seed+round(rnorm(1,0,100))
  
  for(w in 1:WW)
  {
    for(t in 1:TT)
    {
      #set.seed(seed)
      random=rbinom(1,1,ps[t])
      for (i in 1:Nbar) 
      {
        Mmat[(w-1)*Tmax+t,(i-1)*TT+t]=random
      }
    }
    #seed=seed+round(rnorm(1,0,100))
  }
  return(Mmat)
}

> pvecf(ps,2,10)
[1] 8 8 8 8

In [2]:
pvecf=function(ps,Nbar,ci) #ci:unit idle cost
{
  pvec=ps #set noshow probability per slot
  for(i in 2:Nbar)
  {
    pvec=c(pvec,ps)
  }
  pvec=ci*pvec
  return(pvec)
}#Nbar*T维度，每个值都是show的概率*ci，是目标函数中z的系数，未乘W

lambda=c(0.1,0.3,0.5,0.7,0.9,0.9,0.7,0.5,0.3,0.1)
> vvecf(10,lambda)
 [1] 1 3 5 7 9 9 7 5 3 1

In [2]:
vvecf=function(ci,lambda)
{
  vec=lambda*ci
  return(vec)
}#T维度，是ci*γ的期望值，是目标函数中v的系数，未乘W

> csvecf(0.8,2,2)
[1] 0.1 0.1 0.1 0.1

In [2]:
csvecf=function(cw,Tmax,W) #unit waiting cost for walk-ins
{
  cvec=rep((1-cw)/W,Tmax*W)  
  return(cvec)
}

> cfvecf(cn,cw,2,2)
[1] 0.35 0.35 0.35 0.35

In [2]:
cfvecf=function(cn,cw,Tmax,W) #unit waiting cost for telemedical patients
{
  cvec=rep((cw-cn)/W,Tmax*W)
  return(cvec)
}

In [2]:
Vmatf=function(Tmax,TT,WW,lambda2,seed,dvec,d0vec)
{
  set.seed(seed)
  Vmat=matrix(0,Tmax*WW,TT)
  for (w in 1:WW) {
    for (t in 1:TT) {
      Vmat[(w-1)*Tmax+t,t]=d0vec[(w-1)*Tmax+t]-dvec[(w-1)*Tmax+t]
    }
  }
  return(Vmat)
}

> c3vecf(cn,2,2)
[1] 0.05 0.05 0.05 0.05

In [2]:
c3vecf=function(cn,Tmax,W)
{
  cvec=rep(cn/W,Tmax*W)  
  return(cvec)
}

> c0vecf(cd,2,2,2)
[1]  0.00001 12.50000  0.00001 12.50000

In [2]:
c0vecf=function(cd,Tmax,W,T) #overtime+idle cost
{
  cvec=rep(0.00001,Tmax*W)
  for (w in 1:W) {
    cvec[(w-1)*Tmax+T]=cd/W
  }
  return(cvec)
}

> dvecf(2,2,2,lambda,1)
[1] -1 -1 -1  0

In [2]:
dvecf=function(Tmax,W,T,lambda,seed)
{
  set.seed(seed)
  dvec=rep(-1,Tmax*W)
  for(w in 1:W)
  {
    for(t in 1:T)
    {
      dvec[(w-1)*Tmax+t]=dvec[(w-1)*Tmax+t]+rpois(1,lambda[t])
    }
  }  
  return(dvec)
}

#Generate d vector (sample based)
dvec=dvecf(Tmax,W,T,lambda,round(rnorm(1,0,100)))
> d0vecf(2,2,2,lambda,1,dvec)
[1] -1 -1 -1  1

In [2]:
d0vecf=function(Tmax,T,W,lambda,seed,dvec)
{
  set.seed(seed)
  for(w in 1:W)
  {
    for(t in 1:T)
    {
      dvec[(w-1)*Tmax+t]=dvec[(w-1)*Tmax+t]+rpois(1,lambda[t])
    }
  }
  return(dvec)
}

#unused

In [2]:
if(FALSE)
{
walkinzip=function(lambda,pzero,T,Wmax)
{
  t=1
  p=matrix(0,T,Wmax+1)
  while(t<=T)
  {
    p[t,Wmax+1]=1
    p[t,1]=pzero[t]+(1-pzero[t])*dpois(0,lambda[t],FALSE)
    p[t,Wmax+1]=p[t,Wmax+1]-p[t,1]
    i=1
    while(i<Wmax)
    {
      p[t,i+1]=(1-pzero[t])*dpois(i,lambda[t],FALSE)
      p[t,Wmax+1]=p[t,Wmax+1]-p[t,i+1]
      i=i+1
    }
    t=t+1
  }
  return(p)
}
}

In [3]:
mipfull=function(WW,Tmax,Nbar,TT,Umat,U1mat,U2mat,Emat,E2mat,Vmat,Mmat,csvec,cfvec,c3vec,c0vec,pvec,vvec,dvec,d0vec,Xmax)
{
  cvec=rep(0.00001,Tmax*WW)
  cpvec=c(csvec,cfvec,c3vec,cvec,c0vec,-pvec,-vvec)
  
  evec=rep(1,Nbar)
  
  O1mat=sparseMatrix(i=1,j=1,x=0,dims=c(Nbar,TT))
  O2mat=sparseMatrix(i=1,j=1,x=0,dims=c(Nbar,WW*Tmax))
  O3mat=sparseMatrix(i=1,j=1,x=0,dims=c(Tmax*WW,TT))
  O4mat=sparseMatrix(i=1,j=1,x=0,dims=c(WW*Tmax,WW*Tmax))
  O5mat=sparseMatrix(i=1,j=1,x=0,dims=c(TT,WW*Tmax))
  O6mat=sparseMatrix(i=1,j=1,x=0,dims=c(TT,TT))
  
  OE2mat=cbind(O5mat,O5mat,O5mat,O5mat,O5mat,E2mat,O6mat)
  OEmat=cbind(O2mat,O2mat,O2mat,O2mat,O2mat,Emat,O1mat)
  
  UMmats=cbind(-Umat,O4mat,O4mat,O4mat,O4mat,Mmat,O3mat)
  UMmatf=cbind(O4mat,-Umat,O4mat,O4mat,O4mat,Mmat,O3mat)
  UMmat3=cbind(O4mat,O4mat,U1mat,U2mat,O4mat,Mmat,O3mat)
  UMmat2=cbind(O4mat,O4mat,O4mat,U1mat,U2mat,Mmat,O3mat)
  UMmat1=cbind(O4mat,O4mat,O4mat,O4mat,-Umat,Mmat,Vmat)
  
  
  
  AAmat=rbind(OE2mat,OEmat,UMmats,UMmatf,UMmat3,UMmat2,UMmat1)
  
  rhsvec=c(Xmax,evec,-dsvec,rep(-dvec,4))
  
  dir=rep("<=",5*Tmax*WW+Nbar+TT)
  
  #[ 0  0  0  0  0  E2  0] ys <=  Xmax   
  #[ O  O  0  0  0  E  0]  yf <=  e      
  #[-U  O  0  0  0  M  0]  y3 <= -ds    
  #[ O -U  0  0  0  M  0]  y2 <= -df     
  #[ O  0  U1 U2 0  M  0]  y1 <= -d3
  #[ O  0  0  U1 U2 M  0]  z  <= -d2
  #[ O  0  0  0 -U  M  V]  v  <= -d1
  
  types=c(rep("I",5*Tmax*WW),rep("B",Nbar*TT+TT))
  
  
  
  
  model <- list()
  
  model$A          <- AAmat
  model$obj        <- cpvec
  model$modelsense <- "min"
  model$rhs        <- rhsvec
  model$sense      <- dir
  model$vtype      <- types
  
  params <- list(OutputFlag=0,Method=2)
  
  result <- gurobi(model, params)
  
  
  #print('Solution:')
  print(result$objval)
  #print(result$x)
  zstar=result$x
  #zstar=sol$solution
  i=c()
  j=c()
  x=c()
  for(n in 1:Nbar)
    for(t in 1:TT)
    {
      i=c(i,t)
      j=c(j,5*Tmax*WW+(n-1)*TT+t)
      x=c(x,1)
    } #i是1,2,...,TT,1,2...,TT,Nbar个
  #j是5*Tmax*WW+1,5*Tmax*WW+2,...,5*Tmax*WW+TT+1,...,5*Tmax*WW+(Nbar-1)TT+1,...
  
  outmat=sparseMatrix(i,j,x=x,dims=c(TT,5*WW*Tmax+Nbar*TT+TT))
  output=as.vector(outmat%*%zstar) #得到x
  a=5*WW*Tmax+Nbar*TT+TT
  b=a-TT+1
  output=c(output,zstar[b:a]) #把v加上去
  return(output)
  #return(zstar)
}


In [4]:
#考虑线上患者，但不考虑v
mip1=function(WW,Tmax,Nbar,TT,Umat,U1mat,U2mat,Emat,E2mat,Mmat,csvec,cfvec,c3vec,c0vec,pvec,dvec,d0vec,Xmax)
{
  cvec=rep(0.00001,Tmax*WW)
  
  cpvec=c(csvec,cfvec,c3vec,cvec,c0vec,-pvec)
  
  evec=rep(1,Nbar)
  
  #O1mat=sparseMatrix(i=1,j=1,x=0,dims=c(Nbar,TT))
  O2mat=sparseMatrix(i=1,j=1,x=0,dims=c(Nbar,WW*Tmax))
  #O3mat=sparseMatrix(i=1,j=1,x=0,dims=c(Tmax*WW,TT))
  O4mat=sparseMatrix(i=1,j=1,x=0,dims=c(WW*Tmax,WW*Tmax))
  O5mat=sparseMatrix(i=1,j=1,x=0,dims=c(TT,WW*Tmax))
  #O6mat=sparseMatrix(i=1,j=1,x=0,dims=c(TT,TT))
  
  OE2mat=cbind(O5mat,O5mat,O5mat,O5mat,O5mat,E2mat)
  OEmat=cbind(O2mat,O2mat,O2mat,O2mat,O2mat,Emat)
  
  UMmats=cbind(-Umat,O4mat,O4mat,O4mat,O4mat,Mmat)
  UMmatf=cbind(O4mat,-Umat,O4mat,O4mat,O4mat,Mmat)
  UMmat3=cbind(O4mat,O4mat,U1mat,U2mat,O4mat,Mmat)
  UMmat2=cbind(O4mat,O4mat,O4mat,U1mat,U2mat,Mmat)
  UMmat1=cbind(O4mat,O4mat,O4mat,O4mat,-Umat,Mmat)
  
  
  
  AAmat=rbind(OE2mat,OEmat,UMmats,UMmatf,UMmat3,UMmat2,UMmat1)
  
  rhsvec=c(Xmax,evec,-dsvec,rep(-dvec,3),-d0vec)
  
  dir=rep("<=",5*Tmax*WW+Nbar+TT)
  
  #[ 0  0  0  0  0  E2 ] ys <=  Xmax   
  #[ O  O  0  0  0  E  ]  yf <=  e      
  #[-U  O  0  0  0  M  ]  y3 <= -ds    
  #[ O -U  0  0  0  M  ]  y2 <= -df     
  #[ O  0  U1 U2 0  M  ]  y1 <= -d3
  #[ O  0  0  U1 U2 M  ]  z  <= -d2
  #[ O  0  0  0 -U  M  ]    <= -d1
  
  types=c(rep("I",5*Tmax*WW),rep("B",Nbar*TT))
  
  
  
  
  model <- list()
  
  model$A          <- AAmat
  model$obj        <- cpvec
  model$modelsense <- "min"
  model$rhs        <- rhsvec
  model$sense      <- dir
  model$vtype      <- types
  
  params <- list(OutputFlag=0,Method=2)
  
  result <- gurobi(model, params)
  
  
  
  zstar=result$x
  
  i=c()
  j=c()
  x=c()
  for(n in 1:Nbar)
    for(t in 1:TT)
    {
      i=c(i,t)
      j=c(j,5*Tmax*WW+(n-1)*TT+t)
      x=c(x,1)
    } #i是1,2,...,TT,1,2...,TT,Nbar个
  #j是6*Tmax*WW+1,6*Tmax*WW+2,...,6*Tmax*WW+TT+1,...,6*Tmax*WW+(Nbar-1)TT+1,...
  
  outmat=sparseMatrix(i,j,x=x,dims=c(TT,5*WW*Tmax+Nbar*TT))
  output=as.vector(outmat%*%zstar)
  #output=c(zstar[1:(5*WW*Tmax)],Mmat%*%(zstar[(5*WW*Tmax+1):(5*WW*Tmax+Nbar*TT)]),output, zstar[(5*WW*Tmax+1):(5*WW*Tmax+Nbar*TT)])
  #return(result)
  #print(result$objval)
  return(output)
  
}

In [5]:
## recursive local search
uijmatf=function(TT)
{
  uij=matrix(0,TT+1,TT)
  uij[1,1]=-1
  uij[TT+1,TT]=1
  
  i=2
  while(i<=TT)
  {
    uij[i,i-1]=1
    uij[i,i]=-1
    i=i+1
  }
  
  return(uij)
}
## 为-1的下标：(1,1)(2,2)(3,3)...(TT,TT)
## 为1的下标：(2,1)(3,2)...(TT,TT-1)(TT+1,TT)


showprop=function(ps,x,TT,Tmax) #x是xini 初始的安排x
{
  px=matrix(0,Tmax,max(x)+2) #这个为什么是+2？
  for(t in 1:TT)
  {
    for(n in 0:x[t])
    {
      px[t,n+1]=dbinom(n,x[t],ps[t],FALSE)
    }
  }
  t=TT+1
  while (t<=Tmax) {
    px[t,1]=1
    t=t+1
  }
  return(px)
}
# (t,n)的数值为第t时刻来n个scheduled病人的概率 

In [6]:
walkinpoi=function(lambda,TT,Wmax,Tmax)
{
  t=1
  p=matrix(0,Tmax,Wmax+2)
  while(t<=TT)
  {
    i=0
    p[t,Wmax+2]=1
    while(i<(Wmax+1))
    {
      p[t,i+1]=dpois(i,lambda[t],FALSE)  #dpois，给出x处的概率密度
      p[t,Wmax+2]=p[t,Wmax+2]-p[t,i+1]   #每行代表第t个时间，第i列代表来i-1个人的概率，
      #第Wmax+2列应该是除了前面的可能性，来更多人的可能性
      i=i+1
    }
    t=t+1
  }
  t=TT+1
  while (t<=Tmax) {
    p[t,1]=1
    t=t+1
  }
  return(p)
}

In [7]:
onlinepoi=function(lambda2,TT,Lmax,Tmax)
{
  t=1
  p=matrix(0,Tmax,Lmax+2)
  while(t<=TT)
  {
    i=0
    p[t,Lmax+2]=1
    while(i<(Lmax+1))
    {
      p[t,i+1]=dpois(i,lambda2[t],FALSE)  #dpois，给出x处的概率密度
      p[t,Lmax+2]=p[t,Lmax+2]-p[t,i+1]   #每行代表第t个时间，第i列代表来i-1个人的概率，
      #第Wmax+2列应该是除了前面的可能性，来更多人的可能性
      i=i+1
    }
    t=t+1
  }
  t=TT+1
  while (t<=Tmax) {
    p[t,1]=1
    t=t+1
  }
  return(p)
}

In [8]:
onlinepoiv=function(lambda2,TT,Lmax,Tmax,v)
{
  t=1
  p=matrix(0,Tmax,Lmax+2)
  while(t<=TT)
  {
    if(v[t]==1){
      i=0
      p[t,Lmax+2]=1
      while(i<(Lmax+1))
      {
        p[t,i+1]=dpois(i,lambda2[t],FALSE)  #dpois，给出x处的概率密度
        p[t,Lmax+2]=p[t,Lmax+2]-p[t,i+1]   #每行代表第t个时间，第i列代表来i-1个人的概率，
        #第Wmax+2列应该是除了前面的可能性，来更多人的可能性
        i=i+1
      }}
    else{
      p[t,1]=1
    }
    t=t+1
  }
  t=TT+1
  while (t<=Tmax) {
    p[t,1]=1
    t=t+1
  }
  return(p)
}

In [9]:
recursivefull=function(Tmax,p,px,q,Wmax,Lmax,Nmax,xs)
{
  ppt=matrix(0,Tmax,Nmax+1)    #Yt(k)
  pptx=matrix(0,Tmax,Nmax+1)  #Xt(k)
  ppt1=matrix(0,Tmax,Nmax+1)  #Zt1(k)
  ppt2=matrix(0,Tmax,Nmax+1)  #Zt2(k)
  ppt3=matrix(0,Tmax,Nmax+1)  #Zt3(k)
  
  
  ###先算第一格
  ppt[1,1]=px[1,1]*(p[1,1]+p[1,2])+px[1,2]*p[1,1] # p是walkinpoi函数构造的；px是showprop构造的
  #ppt[t,n]是t时刻末有n-1人在等待的概率
  pptx[1,1]=px[1,1]+px[1,2]
  #pptx[t,n]是t时刻末有n-1个scheduled病人在等待的概率
  
  # 线下+等待了1个及以上slot的线上人数等于0的概率：
  ppt1[1,1]=px[1,2]*p[1,1]*q[1,1]+px[1,1]*(p[1,2]*q[1,1]+p[1,1]*q[1,2]+p[1,1]*q[1,1])
  
  ppt2[1,1]=ppt[1,1]
  ppt3[1,1]=ppt[1,1]
  
  
  ### 算t=1时的每个概率
  # ppt1
  n=1
  while(n<=Nmax) # Nmax：maximum number of all patients
  {
    xi=0
    while(xi<=xs[1]) #xs是什么？可能是初始schedule
    {
      j=0
      while (j<=n-xi+1) #j是walkin的数量，xi是scheduled数量，n-xi-j+1是online的数量
      {
        if( j>=0 && j<=Wmax && (n-xi-j+1)>=0 && (n-xi-j+1)<=Lmax)
        {
          ppt1[1,n+1]=ppt1[1,n+1]+px[1,xi+1]*p[1,j+1]*q[1,n-xi-j+1+1]
        } # Wmax：maximum number of walkins per slot
        j=j+1
      }
      
      xi=xi+1
    }
    n=n+1
  }
  
  # ppt
  n=1
  while(n<=Nmax) # Nmax：maximum number of all patients
  {
    xi=0
    while(xi<=xs[1]) 
    {
      if((n-xi+1)>=0 && (n-xi+1)<=Wmax)
        { # Wmax：maximum number of walkins per slot
        ppt[1,n+1]=ppt[1,n+1]+px[1,xi+1]*p[1,n-xi+1+1]}
        xi=xi+1
    }
    ppt2[1,n+1]=ppt[1,n+1]
    ppt3[1,n+1]=ppt[1,n+1]
    n=n+1
  }
  
  #pptx
  n=1
  while((n+1)<=xs[1])
  {
    pptx[1,n+1]=pptx[1,n+1]+px[1,n+2]
    n=n+1
  }
  
  
  
  # he=rep(0,Tmax)
  # for (j in 1:Tmax) {
  #   for (i in 1:Nmax)
  #   {
  #     he[j]=he[j]+ppt[j,i]
  #   }
  # }
  # print(he)
  
  ### 算其他t的，先算ppt，pptx，pptt
  t=2
  
  while(t<=Tmax)
  {
    #算k=0的情况
    ppt[t,1]=ppt[t-1,1]*(px[t,1]*(p[t,1]+p[t,2])+px[t,2]*p[t,1])+ppt[t-1,2]*px[t,1]*p[t,1]
    pptx[t,1]=pptx[t-1,1]*(px[t,1]+px[t,2])+pptx[t-1,2]*px[t,1]
    ppt1[t,1]=ppt1[t-1,1]*(px[t,1]*(p[t,1]*(q[t,1]+q[t,2])+p[t,2]*q[t,1])+px[t,2]*p[t,1]*q[t,1])+ppt1[t-1,2]*px[t,1]*p[t,1]*q[t,1]
    ppt2[t,1]=ppt1[t-1,1]*(px[t,1]*(p[t,1]+p[t,2])+px[t,2]*p[t,1])+ppt1[t-1,2]*px[t,1]*p[t,1]
    ppt3[t,1]=ppt2[t-1,1]*(px[t,1]*(p[t,1]+p[t,2])+px[t,2]*p[t,1])+ppt2[t-1,2]*px[t,1]*p[t,1]
    #ppt,k>0的情况
    n=1
    while(n<=Nmax) #Nmax是所有患者的最大人数
    {
      xi=0
      while((t<=TT && xi<=xs[t]) || (t>TT && t<=Tmax && xi<=0))
      {
        i=0
        while(i<=Nmax) #i是上一个时刻末的等待人数
        {
          if((n-i-xi+1)>=0 && (n-i-xi+1)<=Wmax)
            ppt[t,n+1]=ppt[t,n+1]+ppt[t-1,i+1]*px[t,xi+1]*p[t,n-i-xi+1+1]
          i=i+1
        }
        xi=xi+1
      }
      n=n+1
    }
    
    #pptx,k>0的情况
    n=1
    while(n<=Nmax)
    {
      xi=0
      while((t<=TT && xi<=xs[t]) || (t>TT && t<=Tmax && xi<=0))
      {
        if((n-xi+1)<=Nmax && (n-xi+1)>=0)
          pptx[t,n+1]=pptx[t,n+1]+pptx[t-1,n-xi+1+1]*px[t,xi+1]
        xi=xi+1
      }
      n=n+1
    }
    
    
    #ppt1,k>0
    n=1
    while(n<=Nmax) #Nmax是所有患者的最大人数
    {
      xi=0
      while((t<=TT && xi<=xs[t]) || (t>TT && t<=Tmax && xi<=0))
      {
        i=0
        while(i<=Nmax) #i是上一个时刻末的等待人数
        {
          j=0
          while (j<=n-i-xi+1) 
          {
            if( j>=0 && j<=Wmax && (n-xi-i-j+1)>=0 && (n-xi-i-j+1)<=Lmax)
            {
              ppt1[t,n+1]=ppt1[t,n+1]+ppt1[t-1,i+1]*px[t,xi+1]*p[t,j+1]*q[t,n-i-xi-j+1+1]
            }
            j=j+1
          }
          i=i+1
        }
        xi=xi+1
      }
      n=n+1
    }
    
    #ppt2,k>0
    n=1
    while(n<=Nmax) #Nmax是所有患者的最大人数
    {
      xi=0
      while((t<=TT && xi<=xs[t]) || (t>TT && t<=Tmax && xi<=0))
      {
        i=0
        while(i<=Nmax) #i是上一个时刻末的等待人数
        {
          if((n-i-xi+1)>=0 && (n-i-xi+1)<=Wmax)
            ppt2[t,n+1]=ppt2[t,n+1]+ppt1[t-1,i+1]*px[t,xi+1]*p[t,n-i-xi+1+1]
          i=i+1
        }
        xi=xi+1
      }
      n=n+1
    }
    
    
    #ppt3,k>0
    n=1
    while(n<=Nmax) #Nmax是所有患者的最大人数
    {
      xi=0
      while((t<=TT && xi<=xs[t]) || (t>TT && t<=Tmax && xi<=0))
      {
        i=0
        while(i<=Nmax) #i是上一个时刻末的等待人数
        {
          if((n-i-xi+1)>=0 && (n-i-xi+1)<=Wmax)
            ppt3[t,n+1]=ppt3[t,n+1]+ppt2[t-1,i+1]*px[t,xi+1]*p[t,n-i-xi+1+1]
          i=i+1
        }
        xi=xi+1
      }
      n=n+1
    }
    

    
    
    t=t+1
  }
  
  
  
  
  expx=rep(0,Tmax)
  exp=rep(0,Tmax)
  exp1=rep(0,Tmax)
  exp2=rep(0,Tmax)
  exp3=rep(0,Tmax)
  
  
  for(t in 1:Tmax)
  {
    exp[t]=0
    expx[t]=0
    exp1[t]=0
    exp2[t]=0
    exp3[t]=0
    for(i in 1:Nmax)
    {
      exp[t]=exp[t]+i*ppt[t,i+1]
      expx[t]=expx[t]+i*pptx[t,i+1]
      exp1[t]=exp1[t]+i*ppt1[t,i+1]
      exp2[t]=exp2[t]+i*ppt2[t,i+1]
      exp3[t]=exp3[t]+i*ppt3[t,i+1]
    }
  } #算每个t末等待人数的期望值
  
  # exp[Tmax+1]=0
  # expx[Tmax+1]=0
  
  # for(i in 2:Nmax)
  # {
  #   exp[TT+1]=exp[TT+1]+i*(i-1)*0.5*ppt[TT,i+1]
  #   expx[TT+1]=expx[TT+1]+i*(i-1)*0.5*pptx[TT,i+1]
  # } #TT时刻等待时间期望值，算的是TT+1及以后所有时间段的
  
  output<-list(op=exp,opx=expx,op1=exp1,op2=exp2,op3=exp3)
  return(output)
  
}


In [10]:
calculatefull=function(TT,Tmax,exp,expx,exp1,exp3,co,ci,cw,cn,x,mean,mean2,ps)
{
  overtime=exp1[TT]
  idletime=exp1[TT]+TT-sum(x*ps)-sum(mean)-sum(mean2) #mean是每个时刻walkin的均值
  waittime=sum(exp)
  waittimex=sum(expx)
  waittime3=sum(exp3)-sum(exp)
  cost=cw*(waittime-waittimex)+waittimex+cn*(waittime3)+co*overtime+ci*idletime
  
  return(cost)
  
}


calculatefullv=function(TT,Tmax,exp,expx,exp1,exp3,co,ci,cw,cn,x,mean,mean2,ps,v)
{
  overtime=exp1[TT]
  idletime=exp1[TT]+TT-sum(x*ps)-sum(mean)-sum(mean2*v) #mean是每个时刻walkin的均值
  waittime=sum(exp)
  waittimex=sum(expx)
  waittime3=sum(exp3)-sum(exp)
  cost=cw*(waittime-waittimex)+waittimex+cn*(waittime3)+co*overtime+ci*idletime
  
  return(cost)
  
}



valid=function(x,TT,Xmax)
{
  isvalid=1
  for(i in 1:TT)
  {
    if(x[i]<0||x[i]>Xmax[i])
      isvalid=0
  }
  return(isvalid)
}


localsearchfull=function(TT,Tmax,meanwi,mean2,ps,Xmax,p,Wmax,Nmax,Lmax,co,ci,cw,cn,xini)
{
  isopt=0
  x=xini
  
  
  uij=uijmatf(TT)
  
  #vij=vijmat(TT)
  kk=2^(TT+1)-2 #为什么是-2？
  #jj=1
  while(isopt==0)
  {
    isopt=1
    px=showprop(ps,x,TT,Tmax) #px是每个t来n个scheduled病人的概率分布
    output1=recursivefull(Tmax,p,px,q,Wmax,Lmax,Nmax,x)
    exp1=output1$op #所有人
    expx1=output1$opx #scheduled人
    exp31=output1$op3
    exp11=output1$op1
    cost1=calculatefull(TT,Tmax,exp1,expx1,exp11,exp31,co,ci,cw,cn,x,meanwi,mean2,ps)
    
    i=1
    
    while(i <=kk)
    {
      a=intToBits(i) #这个函数是啥
      a=as.vector(a,"numeric")
      a=a[1:(TT+1)]
      x2=x+as.vector(a%*%uij) #x2是x1的neighbor
      if(valid(x2,TT,Xmax)==1) #检查x2是否满足每个分量小于Xmax
      {
        #jj=jj+1
        px2=showprop(ps,x2,TT,Tmax)
        output2=recursivefull(Tmax,p,px2,q,Wmax,Lmax,Nmax,x2)
        exp2=output2$op
        expx2=output2$opx
        exp32=output2$op3
        exp12=output2$op1
        
        cost2=calculatefull(TT,Tmax,exp2,expx2,exp12,exp32,co,ci,cw,cn,x2,meanwi,mean2,ps)
        if(cost2<cost1)
        {
          x=x2
          isopt=0
          cost1=cost2
          break
        }
      }
      i=i+1
    }
  }
  #print(jj)
  return(x)
}

In [11]:
# LS+LP
lpfull=function(WW,Tmax,Nbar,TT,Umat,Emat,cfvec,csvec,c3vec,c0vec,pvec,Mmat,dvec,dsvec,d0vec)
{
  cvec=rep(0,Tmax*WW)
  cpvec=c(csvec,cfvec,c3vec,cvec,c0vec,-pvec)
  
  evec=rep(1,Nbar)
  
  Omat=sparseMatrix(i=1,j=1,x=0,dims=c(Nbar,WW*Tmax))
  OEmat=cbind(Omat,Omat,Omat,Omat,Omat,Emat)
  
  Ommat=sparseMatrix(i=1,j=1,x=0,dims=c(WW*Tmax,WW*Tmax))
  UMmat=rbind(
    cbind(-Umat,Ommat,Ommat,Ommat,Ommat,Mmat),cbind(Ommat,-Umat,Ommat,Ommat,Ommat,Mmat),
    cbind(Ommat,Ommat,-Umat,Ommat,Ommat,Mmat),cbind(Ommat,Ommat,Ommat,-Umat,Ommat,Mmat),
    cbind(Ommat,Ommat,Ommat,Ommat,-Umat,Mmat))
  
  AAmat=rbind(OEmat,UMmat)
  
  edvec=c(evec,-dsvec,rep(-dvec,3),-d0vec)
  
  dir=rep("<=",5*Tmax*WW+Nbar)
  
  #                              
  #[ O  O  0  0  0  E  ]  ys <=  e      
  #[-U  O  0  0  0  M  ]  yf <= -ds    
  #[ O -U  0  0  0  M  ]  y3 <= -df     
  #[ O  0 -U  0  0  M  ]  y2 <= -d3
  #[ O  0  0 -U  0  M  ]  y1 <= -d2
  #[ O  0  0  0 -U  M  ]  z  <= -d1
  
  
  types=c(rep("C",5*Tmax*WW),rep("C",Nbar*TT))
  
  
  model <- list()
  
  model$A          <- AAmat
  model$obj        <- cpvec
  model$modelsense <- "min"
  model$rhs        <- edvec
  model$sense      <- dir
  model$vtype      <- types
  
  params <- list(OutputFlag=0,Method=2)
  
  result <- gurobi(model, params)
  
  #print('Solution:')
  #print(result$objval)
  #print(result$x)
  zstar=result$x
  #zstar=sol$solution
  i=c()
  j=c()
  x=c()
  for(n in 1:Nbar)
    for(t in 1:TT)
    {
      i=c(i,t)
      j=c(j,5*Tmax*WW+(n-1)*TT+t)
      x=c(x,1)
    }
  outmat=sparseMatrix(i,j,x=x,dims=c(TT,5*WW*Tmax+Nbar*TT))
  output=as.vector(outmat%*%zstar)
  return(output)
} #先relaxed lp求初值


initialize=function(x,TT)
{
  for(t in 1:TT)
  {
    x[t]=round(x[t])
  }
  return(x)
}


xtoz=function(x,TT,Nbar)
{
  z=rep(0,Nbar*TT)
  k=0
  for(t in 1:TT)
  {
    if(x[t]>0)
    {
      for(i in 1:x[t])
      {
        z[k*TT+t]=1
        k=k+1
      }
    }
  }
  return(z)
} #把x转化成z

In [12]:
primfull=function(WW,Tmax,cfvec,csvec,c3vec,c0vec,pvec,Umat,U1mat,U2mat,Mmat,dvec,dsvec,d0vec,dir,types,z)
{
  cvec=rep(0.00001,Tmax*WW)
  #obj=csvec*ys+cfvec*yf+c3vec*y3+cvec*y2+c0vec*y1
  
  #[-U  O  0  0  0 ]  ys <= -Mz-ds    
  #[ O -U  0  0  0 ]  yf <= -Mz-df     
  #[ O  0  U1 U2 0 ]  y3 <= -Mz-d3
  #[ O  0  0  U1 U2]  y2 <= -Mz-d2
  #[ O  0  0  0 -U ]  y1 <= -Mz-d0  Mz放在右边因为z不是决策变量了
  
  rhss=Mmat%*%z+dsvec
  rhsf=Mmat%*%z+dvec
  rhs3=Mmat%*%z+dvec
  rhs2=Mmat%*%z+dvec
  rhs1=Mmat%*%z+d0vec
  rhs=c(rhss,rhsf,rhs3,rhs2,rhs1)
  
  Omat=sparseMatrix(i=1,j=1,x=0,dims=c(WW*Tmax,WW*Tmax))
  AAmat=rbind(cbind(-Umat,Omat,Omat,Omat,Omat),cbind(Omat,-Umat,Omat,Omat,Omat),
              cbind(Omat,Omat,U1mat,U2mat,Omat),cbind(Omat,Omat,Omat,U1mat,U2mat),
              cbind(Omat,Omat,Omat,Omat,-Umat))
  cvec=c(csvec,cfvec,c3vec,cvec,c0vec)
  #solver 1
  #sol=Rglpk_solve_LP(cvec, -Umat, dir, -rhs, bounds, types, FALSE)
  
  #solver2
  model <- list()
  model$A          <- AAmat
  model$obj        <- cvec
  model$modelsense <- "min"
  model$rhs        <- -rhs
  model$sense      <- dir
  model$vtype      <- types
  params <- list(OutputFlag=0, Method=2)
  
  result <- gurobi(model, params)
  
  obj=result$objval-pvec%*%z
  #obj=c(result$x,obj)
  return(obj)
  #return(result)
} #给定一个z，求y的最优解，返回目标函数值

# ys=obj[1:(WW*Tmax)]
# yf=obj[(WW*Tmax+1):(2*WW*Tmax)]
# yf3=obj[(2*WW*Tmax+1):(3*WW*Tmax)]
# yf2=obj[(3*WW*Tmax+1):(4*WW*Tmax)]
# yf1=obj[(4*WW*Tmax+1):(5*WW*Tmax)]
# 
# ys
# yf
# yf3
# yf2
# yf1

In [13]:
# 下面那个函数，是local search z，然后给定z算每个cost，选最小的那个
localzpf=function(TT,Nbar,WW,Tmax,Umat,U1mat,U2mat,cfvec,csvec,c0vec,pvec,dir,types,x,Mmat,dvec,dsvec,dfvec,d0vec,Xmax)
{
  #generate constant matrix,vector
  dir=c(dir,dir,dir,dir,dir)
  types=c(types,types,types,types,types)
  isopt=0
  
  z=xtoz(x,TT,Nbar)
  
  obj=primfull(WW,Tmax,cfvec,csvec,c3vec,c0vec,pvec,Umat,U1mat,U2mat,Mmat,dvec,dsvec,d0vec,dir,types,z)
  objopt=obj
  
  uij=uijmatf(TT)
  #vij=vijmat(TT)
  kk=2^(TT+1)-2 #这个是因为全不选和全选是一样的吗？？
  
  #jj=1
  
  while(isopt==0)
  {
    isopt=1
    i=1
    while(i<=kk)
    {
      #x0=x+vij[i,]
      a=intToBits(i)
      a=as.vector(a,"numeric")
      a=a[1:(TT+1)]
      x0=x+as.vector(a%*%uij)
      if(valid(x0,TT,Xmax)==1)
      {
        z0=xtoz(x0,TT,Nbar)
        obj=primfull(WW,Tmax,cfvec,csvec,c3vec,c0vec,pvec,Umat,U1mat,U2mat,Mmat,dvec,dsvec,d0vec,dir,types,z0)
        if(obj<objopt)
        {
          #jj=jj+1
          z=z0
          x=x0
          objopt=obj
          isopt=0
          break
        }
      }
      i=i+1
    }
  }
  print(objopt)
  return(x)
}

In [14]:
#source('2dv-gurobi新.R')



outcome=list()

outcome$optimaltime=list()
outcome$optimal=list()
outcome$optimalcost=c()
outcome$dualtime=list()
outcome$dual=list()
outcome$dualcost=c()
outcome$dualratio=c()
outcome$primtime=list()
outcome$prim=list()
outcome$primcost=c()
outcome$primratio=c()

In [15]:
#input 
TT=10 #number of slots
# # 0.6 vs 0.3
# lambda=c(0.2,0.3,0.3,0.5,0.7,1,1.2,1.2,1,0.7,0.5,0.3,0.3,0.2) #0.6 average walkins per slot
# # lambda2=lambda/2
# #lambda2=c(1.2,1,0.7,0.5,0.3,0.3,0.2,0.2,0.3,0.3,0.5,0.7,1,1.2)/2
# lambda2=rep(0.3,TT)

# 0.5 vs 0.4
# lambda=c(0.2,0.3,0.3,0.5,0.7,1,1.2,1.2,1,0.7,0.5,0.3,0.3,0.2)/6*5
# #lambda2=lambda*4/5
# #lambda2=c(1.2,1,0.7,0.5,0.3,0.3,0.2,0.2,0.3,0.3,0.5,0.7,1,1.2)*2/3
# lambda2=rep(0.4,TT)


# 0.5 vs 0.2
#lambda=c(0.2,0.3,0.3,0.5,0.7,1,1.2,1.2,1,0.7,0.5,0.3,0.3,0.2)/6*5
# lambda2=lambda*2/5
#lambda2=c(1.2,1,0.7,0.5,0.3,0.3,0.2,0.2,0.3,0.3,0.5,0.7,1,1.2)*1/3
#lambda2=rep(0.2,TT)

lambda=c(0.1,0.3,0.5,0.7,0.9,0.9,0.7,0.5,0.3,0.1)
#lambda2=lambda/5*2
#lambda2=rep(0.2,10)
#lambda2=c(0.9,0.7,0.5,0.3,0.1,0.1,0.3,0.5,0.7,0.9)/5*2

#lambda2=lambda/5*4
#lambda2=rep(0.4,10)
lambda2=c(0.9,0.7,0.5,0.3,0.1,0.1,0.3,0.5,0.7,0.9)/5*4


#need to be set for algorithm running
Wmax=ceiling(max(4*lambda)) #maximum number of walkins per slot
Lmax=ceiling(max(7*lambda2))
# 为什么要乘4？

In [16]:
#input
ps=rep(0.8,TT) #set noshow probability per slot

#need to be set for algorithm running
Xmax=ceiling(1/ps+0.1)+1  #maximum number of scheduled patients per slot 

#这个是为什么？

WW=400  #number of senarios for SAA
Nbar=sum(ceiling((1-lambda)/ps))+5 #maximum number of scheduled patients
Tmax=TT+5 #maximum slots to serve all patients 
#为什么加8？ 猜测：因为最多加班一天的0.25倍

#genearate poison distribution for walkins
p=walkinpoi(lambda,TT,Wmax,Tmax)
q=onlinepoi(lambda2,TT,Lmax,Tmax)

Nmax=sum(Xmax)+(Wmax+Lmax)*TT  #maximum number of all patients

In [17]:
#input
ci=10 #idle cost
cd=25 #overtime+idle cost
cw=0.8 #waiting cost for walkins
#cn=0.4
cn=0.1
co=cd-ci #overtime cost

In [18]:
#generate some matrix, signs etc...
{
  dir=rep("<=",Tmax*WW) #milp没用到
  types=rep("I",Tmax*WW) #milp没用到 c是continuous
  Umat=Umatf(Tmax,WW)    #milp用到
  U1mat=U1matf(Tmax,WW)
  U2mat=U2matf(Tmax,WW)
  Emat=Ematf(Nbar,TT)    #milp用到
  E2mat=E2matf(Nbar,TT)  #milp用到
  #Omat=sparseMatrix(i=1,j=1,x=0,dims=c(Nbar,WW*Tmax)) #都没用到
  pvec=pvecf(ps,Nbar,ci) #milp用到
  dsvec=rep(-1,Tmax*WW)  #milp用到
  #c1vec=c1vecf(ci,co,cw,Tmax,WW,TT) #milp用到
  csvec=csvecf(ci,co,cw,Tmax,WW,TT) #milp用到
  cfvec=cfvecf(cn,cw,Tmax,WW)
  c3vec=c3vecf(cn,Tmax,WW)
  c0vec=c0vecf(cd,Tmax,WW,TT)
  set.seed(333)
  #dvec=dvecf(Tmax,WW,TT,lambda,round(rnorm(1,0,100)))
  vvec=vvecf(ci,lambda2)
  set.seed(666)
  #d0vec=d0vecf(Tmax,TT,WW,lambda2,round(rnorm(1,0,100)),dvec)
}

In [19]:
# initialize a schedule
{
  n=round(TT-sum(lambda),0)
  xini=rep(0,TT)
  for(t in 1:n)
  {
    xini[t]=round(1/ps[t],0)
  }
}#xini  milp没用到



{
  ptm <- proc.time()
  xf=localsearchfull(TT,Tmax,lambda,lambda2,ps,Xmax,p,Wmax,Nmax,Lmax,co,ci,cw,cn,xini) #schedule by local search
  proc.time() - ptm
  outcome$optimaltime<-(proc.time() - ptm)
  outcome$optimal<-xf
  xf=round(xf)
  pxf=showprop(ps,xf,TT,Tmax)
  outputf=recursivefull(Tmax,p,pxf,q,Wmax,Lmax,Nmax,xf)
  expf=outputf$op
  expxf=outputf$opx
  exp3f=outputf$op3
  exp1f=outputf$op1
  costf=calculatefull(TT,Tmax,expf,expxf,exp1f,exp3f,co,ci,cw,cn,xf,lambda,lambda2,ps)
  costf
  outcome$optimalcost<-costf #calculate its cost
}#localsearch  using  recursive function

#Generate M matrix (sample based)
set.seed(333)
Mmat=Mmatnsf(Tmax,WW,Nbar,TT,ps,round(rnorm(1,0,100)))
#Generate d vector (sample based)
dvec=dvecf(Tmax,WW,TT,lambda,round(rnorm(1,0,100)))
set.seed(666)
d0vec=d0vecf(Tmax,TT,WW,lambda2,round(rnorm(1,0,100)),dvec)
Vmat=Vmatf(Tmax,TT,WW,lambda2,round(rnorm(1,0,100)),dvec,d0vec)


ptm <- proc.time()
xstarf=mipfull(WW,Tmax,Nbar,TT,Umat,U1mat,U2mat,Emat,E2mat,Vmat,Mmat,csvec,cfvec,c3vec,c0vec,pvec,vvec,dvec,d0vec,Xmax)
proc.time()-ptm
xstarf
xstarf=round(xstarf)
xstarfy=xstarf[1:TT]
v=xstarf[(TT+1):(2*TT)]

xstarfy=c(1,1,1,0,0,0,0,0,0,0)
v=c(1,0,1,0,0,1,0,0,0,0)
pxv=showprop(ps,xstarfy,TT,Tmax)
qv=onlinepoiv(lambda2,TT,Lmax,Tmax,v)
outputv=recursivefull(Tmax,p,pxv,qv,Wmax,Lmax,Nmax,xstarfy)
expv=outputv$op
expxv=outputv$opx
exp3v=outputv$op3
exp1v=outputv$op1
costv=calculatefullv(TT,Tmax,expv,expxv,exp1v,exp3v,co,ci,cw,cn,xstarfy,lambda,lambda2,ps,v)
costv



ptm <- proc.time()
xstarf1=mip1(WW,Tmax,Nbar,TT,Umat,U1mat,U2mat,Emat,E2mat,Mmat,csvec,cfvec,c3vec,c0vec,pvec,dvec,d0vec,Xmax)
proc.time()-ptm
xstarf1
xf=round(xstarf1)
pxf=showprop(ps,xf,TT,Tmax)
outputf=recursivefull(Tmax,p,pxf,q,Wmax,Lmax,Nmax,xf)
expf=outputf$op
expxf=outputf$opx
exp2f=outputf$op2
exptf=outputf$opt
costf=calculatefull(TT,Tmax,expf,expxf,exp2f,exptf,co,ci,cw,cn,xf,lambda,lambda2,ps)
costf

[1] -16.86556


   user  system elapsed 
  39.59    6.03   13.19 

[1] 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0

[1] 32.15501

   user  system elapsed 
   3.20    0.73    2.33 

[1] 1 1 0 0 0 0 0 0 0 0

[1] 27.68581

In [20]:
#generate initial solution using ralaxed LP
xinio=lpfull(WW,Tmax,Nbar,TT,Umat,U1mat,U2mat,Emat,cfvec,csvec,c3vec,c0vec,pvec,Mmat,dvec,dsvec,d0vec)
xini=initialize(xinio,TT)
if(all(xini==xinio)) #这个什么意思？
  xdf=xini

ERROR: Error in lpfull(WW, Tmax, Nbar, TT, Umat, U1mat, U2mat, Emat, cfvec, csvec, : unused arguments (dsvec, d0vec)


In [ ]:
#local search + prim model
ptm <- proc.time()
xpf=localzpf(TT,Nbar,WW,Tmax,Umat,U1mat,U2mat,cfvec,csvec,c0vec,pvec,dir,types,xini,Mmat,dvec,dsvec,dfvec,d0vec,Xmax)
proc.time() - ptm
xpf
#xpf=localzpf(TT,Nbar,Umat,Usmat,c1vec,csvec,pvec,dir,types,xini,Mmat,dvec,dsvec)
outcome$primtime<-c(outcome$primtime,list(proc.time() - ptm))
outcome$prim<-c(outcome$prim,list(xpf))

xpf=round(xpf)
pxpf=showprop(ps,xpf,TT,Tmax) #
outputpf=recursivefull(Tmax,p,pxpf,q,Wmax,Lmax,Nmax,xpf)
exppf=outputpf$op
expxpf=outputpf$opx
exp2pf=outputpf$op2
exptpf=outputpf$opt
costpf=calculatefull(TT,Tmax,exppf,expxpf,exp2pf,exptpf,co,ci,cw,cn,xpf,lambda,lambda2,ps)
costpf

outcome$primcost<-c(outcome$primcost,costpf)
outcome$primratio<-c(outcome$primratio,(costpf-costf)/costf)

In [ ]:
#using milp + solver
ptm <- proc.time()
xstarf=mipfull(WW,Tmax,Nbar,TT,Umat,Emat,E2mat,Vmat,Mmat,csvec,cfvec,c3vec,c0vec,pvec,vvec,dvec,d0vec,Xmax)
outcome$miptime<-c(outcome$miptime,list(proc.time() - ptm))
outcome$mip<-c(outcome$mip,list(xstarf))
pxstarf=showprop(ps,xstarf[1:TT],TT,Tmax)
outputstarf=recursivefull(TT,p,pxstarf,Wmax,Nmax,xstarf)
expstarf=outputstarf$op1
expxstarf=outputstarf$op2
coststarf=calculatefull(TT,expstarf,expxstarf,co,ci,cw,xstarf,lambda,ps)
outcome$mipcost<-c(outcome$mipcost,coststarf)
outcome$mipratio<-c(outcome$mipratio,(coststarf-costf)/costf)